# Train and Create Doordetect Model
This notebook can be used to train a custom (pre-trained) network and convert it for use on an OAK device.

The code is based on this tutorial: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

## Install Libraries

### numpy
Used for image manupulation

In [ ]:
!pip install numpy

### pillow
Dropin replacement for PIL (Python Image Library)

Used form image manipulation

In [ ]:
!pip install pillow

### tensorflow
Used to train the neural network

In [ ]:
!pip install tensorflow

### tensorflow object detection API
Used to train object detection networks

In [ ]:
!git clone https://github.com/tensorflow/models.git
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .
%cd

Test it

In [ ]:
%cd models/research
!python3 object_detection/builders/model_builder_tf2_test.py
%cd

### OPTIONAL: tensorflow_gpu
Used for training on GPU

In [ ]:
!pip install tensorflow_gpu

## Download Dataset
We are going to use an open door detection datase.

If you want to use another dataset you may need to make changes in later steps (especially in [Prepare the Dataset](#prepare_the_dataset))

In [ ]:
!git clone https://github.com/MiguelARD/DoorDetect-Dataset.git

## Prepare the Dataset
This converts the labels/annotations to a TFRecords file which will be used to train the model using the TF2 object detection API.

_NOTE:_ This is specific to the format used in the DoorDetect-dataset and you may need to alter this if you want to use it with another dataset

In [ ]:
import tensorflow as tf
from object_detection.utils import dataset_util, label_map_util
from PIL import Image
import os
import io
import random
from pathlib import Path

images_path = "DoorDetect-Dataset/images"
labels_path = "DoorDetect-Dataset/labels"
label_map_path = "labelmap.pbtxt"
output_train = "train.tfrecords"
output_eval = "eval.tfrecords"

# Load labels
label_map_dict = label_map_util.get_label_map_dict(label_map_path)
labels = {v: k for k, v in label_map_dict.items()}
"""
Alternatively use hardcoded labels
labels = [
    b"door",
    b"handle",
    b"cabinet door",
    b"refrigerator door",
]
"""

"""
Clamp the input number between minumum and maximum.
"""
def clamp(x, minimum=0, maximum=1):
    return min(max(x, minimum), maximum)

"""
Create a single example from an image and an annotation-file
"""
def create_example(image_path, annotation_path):
    #print(f"Processing {image_path.name}")
    with image_path.open('rb') as image:
        encoded_jpg = image.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)

    filename = image_path.name.encode("utf8")
    id = image_path.stem.encode("utf8")
    width, height = image.size
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes = []
    
    with annotation_path.open('r') as annotations:
        for annotation in annotations:
            # Each image may have 0 or more annotations each of the form
            #   class center_x center_y width height
            # with
            #   class: the class of annotated object. Note: The classes/labels in the files start with 0. However the object detection API reserves 0 as background, so we will add 1
            #   center_x, center_y: the center of the bounding box
            #   width, height: the dimensions of the bounding box
            # All coordinates are normalized to the image size and are between 0 and 1.
            data = annotation.split(" ")
            clss = int(data[0]) + 1
            cx = float(data[1])
            cy = float(data[2])
            w = float(data[3]) / 2
            h = float(data[4]) / 2

            # The TFRecords format uses min and max position as bounding boxes instead of centerpoint & dimensions.
            # So we have to convert them.
            # Also make sure that everything is between 0 and 1.
            xmin = clamp(cx - w)
            xmax = clamp(cx + w)
            ymin = clamp(cy - h)
            ymax = clamp(cy + h)
            
            classes.append(clss)
            xmins.append(xmin)
            xmaxs.append(xmax)
            ymins.append(ymin)
            ymaxs.append(ymax)

            #print(f"\t{labels[clss]}({clss}): ({xmin}, {ymin}) / ({xmax}, {ymax})")
    
    return tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(id),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(b"jpg"),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature([labels[x].encode("utf8") for x in classes]),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))


"""
Iterate over all images and create a records file.
output_path: the path where to write the TFRecords-file
image_paths: a list of image files. The corresponding annotation files are automatically searched in `labels_path`

TODO: Do the association of image and annotation somewhere elses
"""
def write_record(output_path, image_paths):
    writer = tf.io.TFRecordWriter(output_path)
    cnt = 0
    # Iterate over all image_paths and check if there is a corresponding annotation
    # If it exists, create an example and write it to the TFRecord
    for image_path in image_paths:
        id = image_path.stem
        annotation_path = (Path(labels_path) / id).with_suffix(".txt")
        if image_path.is_file() and annotation_path.is_file():
            example = create_example(image_path, annotation_path)
            writer.write(example.SerializeToString())
            cnt += 1
        else:
            print(f"WARNING: Image file without annotation ({id})")

    writer.close()
    print(f"Processed {cnt} file(s)")

"""
Partitions a list randomly into two.
percent items go into the first list and (1-percent) into the second.
"""
def partition(list_in, percent=0.9):
    random.shuffle(list_in)
    pivot = int(percent*len(list_in))
    return list_in[:pivot], list_in[pivot:]


image_paths = list(Path(images_path).iterdir())
train_paths, eval_paths = partition(image_paths)
write_record(output_train, train_paths)
write_record(output_eval, eval_paths)

## Download Pre-Trained model
You may use another model from the [object detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [ ]:
!curl http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -o pretrained_model.tar.gz

### Extract the model

In [ ]:
!tar -xf pretrained_model.tar.gz

## Train the Model
After preparing the dataset (TFRecords-file) and downloading a pretrained model we can finally train it.

### Prepare the output directory

In [ ]:
!mkdir -p trained_model/ssd_mobilenet_v2_fpnlite_320x320
!cp ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.conf trained_model/ssd_mobilenet_v2_fpnlite_320x320

**IMPORTANT:** Before you continue, change the pipeline.config [as shown here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html)

### Start the training process
We will use the `model_main_tf2.py`-script bundled with the object detection API for this.

The command line options are:
- `model_dir`: the directory where the trained model should be written to
- `pipeline_config_path`: the path to the `pipeline.config`-file

In [ ]:
!python3 models/research/object_detection/model_main_tf2.py --model_dir=trained_models/ssd_mobilenet_v2_fpnlite_320x320 --pipeline_config_path=trained_models/ssd_mobilenet_v2_fpnlite_320x320/pipeline.config

### Optionally start tensorboard

This can be used to monitor the training progress

In [ ]:
!tensorboard --logdir trained_models/ssd_mobilenet_v2_fpnlite_320x320
# Add the --bind_all option if you want to access the tensorboard from another machine. This is for example necessary if you execute this in the cloud

You may also want to start an evaluation process to better monitor the progress of your training.

This uses the same script that is used for training. However an additional argument is added:
- `checkpoint_dir`: the directory containing the checkpoints. Usually the same as the `model_dir`

In [ ]:
!python3 models/research/object_detection/model_main_tf2.py --model_dir=trained_models/ssd_mobilenet_v2_fpnlite_320x320 --pipeline_config_path=trained_models/ssd_mobilenet_v2_fpnlite_320x320/pipeline.config --checkpoint_dir=trained_models/ssd_mobilenet_v2_fpnlite_320x320

## Export the model
In order to use the model for inference you may want to export it (freeze it).

This can be done with the `exporter_main_v2.py`-script.
The following options are used:
- `input_type`: The type of the input. We will keep this as `image_tensor`
- `pipeline_config_path`: the path to the `pipeline.config`-file
- `trained_checkpoint_dir`: the directory containing the checkpoints. Usually the same as the `model_dir`
- `output_directory`: where to put the output of this script

In [ ]:
!python3 models/research/object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path trained_models/ssd_mobilenet_v2_fpnlite_320x320/pipeline.config --trained_checkpoint_dir trained_models/ssd_mobilenet_v2_fpnlite_320x320/ --output_directory exported_models/ssd_mobilenet_v2_fpnlite_320x320

After the above script finishes you can use the exported model for inference or you may convert it to a `.blob`-file for use on an OAK-device.